# 2. Hageldatenanalyse

In diesem Notebook werden die Hageldaten, welche im Notebook 0_Hageldaten_Download.ipynb heruntergeladen und vorprozessiert wurden, ausgwertet.

Es handelt sich dabei um Daten zur Anzahl der Hageltage sowie die maximal zu erwartende Hagelkorngrösse in den Sommermonaten (April-September) von 2002 bis 2022. Da Hagel und Hagelkorngrössen nicht flächendeckend am Boden gemessen werden können, wird ein Hageltag und die Grösse aus Radarmessungen abgeleitet. 

Ein Hageltag ist definiert als ein Tag, an dem aus den Radarmessungen auf eine hohe Wahrscheinlichkeit (80%) für Hagel am Boden geschlossen wurde, basierend auf dem Radar-Parameter POH (Probability of Hail).

Analog wird die Grösse ebenfalls aus den Radarmessungen abgeleitet und beschreibt die maximal zu erwartende Hagelkorngrösse auf einem Quadaratkilometer basierend auf dem Radar-Parameter MESHS (Maximum Expected Severe Hail Size). Hagelkorngrössen sind in Klassen von 2 cm - grösser 6 cm in 0.5 cm Schritten angegeben, wobei das untere Ende inklusiv, das obere Ende exklusiv ist.

Das Notebook beinhaltet dabei die folgenden Abschnitte:

* [2.1 Daten einlesen und Vorbearbeitung](#hail_data_readin)

* [2.2 Verteilung Hageltage und Hagelkorngrösse Schweiz](#hail_distribution)

* [2.3 Hagel Trends seit 2002](#hail_trends)

* [2.4 Hagelrisiko nach Monat und Kantone](#hail_canton_month)

In [ ]:
! pip install matplotlib
! pip install plotly
! pip install folium
! pip install geopandas
! pip install ipywidgets
! pip install branca

<a id="hail_data_readin"></a>
## 2.1 Daten einlesen und Vorbearbeitung

In diesem Unterabschnitt werden alle nötigen Packages für das gesamte Notebook geladen und die Hageldaten eingelesen.
Es werden ausserdem einige zusätzliche Dataframes auf Grundlage der Hageldaten erzeugt, welche für spätere Analysen benötigt werden. 

Weiterhin wird ein Shapefile eingelesen, welches Polygonzüge der Schweizer Kantonsgrenzen beinhaltet und zur Visualisierung benötigt wird.

In [ ]:
from pathlib import Path
import pandas as pd
import geopandas as gpd

# Karte darstellen
import folium
from branca.colormap import linear

# interaktive Plots
import ipywidgets as widgets
from ipywidgets import interactive
from IPython.display import display

# Plotting
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "iframe" # or 'colab' or 'iframe' or 'iframe_connected' or 'sphinx_gallery'

### 2.1.1 Einlesen der Hageldaten und der Schweizer Kantonsgrenzen

In [ ]:
# vorbereitete Hageldaten
processed_data_path = Path("./data/processed")
hail_data_file = "haildata_per_month.csv"

# Schweizer Kantone Shape file
shape_path = Path("data") / "Swiss_shape_files"
shape_canton_file = "swissBOUNDARIES3D_1_4_TLM_KANTONSGEBIET.shp"

In [ ]:
# Einlesen des Shapefiles für Schweizer Kantone in ein GeoDataFrame
cantons_gdf = gpd.read_file(shape_path / shape_canton_file)

cantons = {
    "Zürich": "ZH",
    "Bern": "BE",
    "Luzern": "LU",
    "Uri": "UR",
    "Schwyz": "SZ",
    "Obwalden": "OW",
    "Nidwalden": "NW",
    "Glarus": "GL",
    "Zug": "ZG",
    "Fribourg": "FR",
    "Solothurn": "SO",
    "Basel-Stadt": "BS",
    "Basel-Landschaft": "BL",
    "Schaffhausen": "SH",
    "Appenzell Ausserrhoden": "AR",
    "Appenzell Innerrhoden": "AI",
    "St. Gallen": "SG",
    "Graubünden": "GR",
    "Aargau": "AG",
    "Thurgau": "TG",
    "Ticino": "TI",
    "Vaud": "VD",
    "Valais": "VS",
    "Neuchâtel": "NE",
    "Genève": "GE",
    "Jura": "JU"
}
canton_map = cantons_gdf.replace({"NAME": cantons})

# Hageldaten einlesen
df_hail = pd.read_csv(processed_data_path / hail_data_file, sep=",")
df_hail.head()

### 2.1.2 Vorbereitung Hageldaten

In [ ]:
# Definieren einer neuen Spalte 'lat_lon', welche 'latitude' (Breitengrad) und 'longitude' (Längengrad) kombiniert darstellt
df_hail["lat_lon"] = df_hail["lat"].astype(str) + "_" + df_hail["lon"].astype(str)

# Definition einer neuen Spalte 'haildays_times_hailsize', welche das Produkt aus maximal zu erwartender Hagelkorngrösse
# und Anzahl der Hageltage je Monat und Messpunkt multipliziert
df_hail["haildays_times_hailsize"] = df_hail["haildays"] * df_hail["hailsize"]

df_hail.head()

In [ ]:
# Erzeugen eines Dataframes, welches nur Einträge mit 'haildays_times_hailsize' grösser 0.0 enthält
# dieses wird benötigt um die mittlere maximal zu erwartende Hagelkorngrösse je Messpunkt zu bestimmen
df_hailsize = df_hail[df_hail["haildays_times_hailsize"] > 0.0]
df_hailsize.head()

In [ ]:
# Berechnung der Messpunkte pro Kanton (benötigt für Normierungszwecke)
points_per_canton = df_hail.groupby("canton")["lat_lon"].nunique()
points_per_canton = points_per_canton.reset_index()
points_per_canton.head()

In [ ]:
# check ob alles korrekt ist
points_switzerland = points_per_canton["lat_lon"].sum()
print(points_switzerland)
print(sum(~df_hail[["lat", "lon"]].duplicated()))
print(points_per_canton["lat_lon"].sum())
sum(~df_hail[["lat", "lon"]].duplicated()) == points_per_canton["lat_lon"].sum()

<a id="hail_distribution"></a>
## 2.2 Verteilung Hageltage und Hagelkorngrösse Schweiz

Im folgenden Unterabschnitt werden die Verteilung der Hageltage und der Hagelkorngrösse in der Schweiz von 2002 bis 2022 in Leaflet maps visualisiert.

Dabei werden alle Hageltage pro Messpunkt in der Schweiz von 2002 bis 2022 aufsummiert, um häufig von Hagel betroffene Gebiete zu identifizieren.

Zur Visualisierung der Hagelkorngrösse pro Messpunkt wird das Produkt aus Hageltagen pro Monat und maximal zu erwartender Hagelkorngrösse pro Monat über alle Monate und Jahre von 2002 bis 2022 aussummiert und durch die Gesamtzahl der Hageltage je Messpunkt über alle Jahre geteilt.

In [ ]:
df_hail.head(n=2)

### 2.2.1 Summe der Hageltage von 2002 - 2022

In [ ]:
# Aufsummieren aller Hageltage von 2002 bis 2022 pro Messpunkt
df_haildays_sum = df_hail.groupby(["lat", "lon"]).agg(
    haildays_sum = ("haildays", "sum"),
).reset_index()
df_haildays_sum.head()

In [ ]:
# Definition der zur Visualierung nötigen Datenpunkte
latitudes = df_haildays_sum.lat
longitudes = df_haildays_sum.lon
haildaysum = df_haildays_sum.haildays_sum

# Definition einer Colormap zur Visualisierung der Summer der Hageltage
colormap = linear.YlGnBu_09.scale(min(haildaysum), max(haildaysum))

# Erzeugen einer Karte mit Zentrum in der Schweiz
switzerland_map = folium.Map(location=[46.8182, 8.2275], zoom_start=8)

# Hinzufügen eines Punktes für jeden Messpunkt zur Karte
for lat, lon, strength in zip(latitudes, longitudes, haildaysum):
    folium.CircleMarker(
        location=[lat, lon],
        radius=2,
        fill=True,
        fill_opacity=0.7,
        color=None,
        fill_color=colormap(strength),
    ).add_to(switzerland_map)

# Hinzufügen der Colormap zur Karte inklusive Legende
colormap.caption = "Summe der Hageltage von 2002 - 2022"
colormap.add_to(switzerland_map)

# Hinzufügen der Kantonsgrenzen als schwarze Linie zur Karte als GeoJSON layer
folium.GeoJson(
    cantons_gdf,
    name="Cantons",
    style_function=lambda feature: {"fillColor": "none", "color": "black", "weight": 2}
).add_to(switzerland_map)

# Steuerungsebene hinzufügen
folium.LayerControl().add_to(switzerland_map)

# Karte darstellen
switzerland_map

### 2.2.2 Hagelkorngrösse von 2002 - 2022

In [ ]:
# Bestimmung der mittleren maximal zu erwartenden Hagelkorngrösse je Messpunkt von 2002 bis 2022
df_hailsize_sum = df_hailsize.groupby(["lat", "lon"]).agg(
    haildays_sum = ("haildays", "sum"),
    hailsize_sum = ("haildays_times_hailsize", "sum"),
).reset_index()
df_hailsize_sum["hailsize_mean"] = (
    df_hailsize_sum["hailsize_sum"]
    / df_hailsize_sum["haildays_sum"]
)
df_hailsize_sum.head()

In [ ]:
# Definition der zur Visualierung nötigen Datenpunkte
latitudes = df_hailsize_sum.lat
longitudes = df_hailsize_sum.lon
mean_hailsize = df_hailsize_sum.hailsize_mean

# Definition einer Colormap zur Visualisierung der Hagelkorngrösse
colormap = linear.YlOrRd_09.scale(min(mean_hailsize), max(mean_hailsize))

# Erzeugen einer Karte mit Zentrum in der Schweiz
switzerland_map = folium.Map(location=[46.8182, 8.2275], zoom_start=8)

# Hinzufügen eines Punktes für jeden Messpunkt zur Karte
for lat, lon, strength in zip(latitudes, longitudes, mean_hailsize):
    folium.CircleMarker(
        location=[lat, lon],
        radius=2,
        fill=True,
        fill_opacity=0.7,
        color=None,
        fill_color=colormap(strength),
    ).add_to(switzerland_map)

# Hinzufügen der Colormap zur Karte inklusive Legende
colormap.caption = "Mittlere maximal zu erwartende Hagelkorngrösse von 2002 - 2022 in cm"
colormap.add_to(switzerland_map)

# Hinzufügen der Kantonsgrenzen als schwarze Linie zur Karte als GeoJSON layer
folium.GeoJson(
    cantons_gdf,
    name="Cantons",
    style_function=lambda feature: {"fillColor": "none", "color": "black", "weight": 2}
).add_to(switzerland_map)

# Steuerungsebene hinzufügen
folium.LayerControl().add_to(switzerland_map)

# Karte darstellen
switzerland_map

<a id="hail_trends"></a>
## 2.3 Hagel Trends seit 2002

Im folgenden Unterabschnitt werden die Hagel Trends seit 2002 (bis einschliesslich 2022) visualisiert. Dabei werden einerseits die Summe der Hageltage pro Jahr und Kanton normiert auf die Anzahl der Messpunkte pro Kanton und andererseits die mittlere maximal zu erwartende Hagelkorngrösse pro Jahr und Kanton an Hageltagen visualisiert.

In [ ]:
# Farbpaletten Defintion für Hageltage
color_map_haildays = "YlGnBu"

# Farbpaletten Defintion für Hagelkorngrösse
color_map_hailsize = "YlOrRd"

# Jahre von 2002 bis 2022
years = df_hail.year.unique()

### 2.3.1 Anzahl der Hageltage pro Jahr und Kanton

In [ ]:
# Gruppieren der Hageldaten nach Jahr und Kanton und Aufsummierung der Hageltage
df_hail_canton = df_hail.groupby(["year", "canton"]).agg(
    haildays_sum = ("haildays", "sum"),
).reset_index()

# Zusammenführen der Hageldaten nach Jahr und Kanton mit Anzahl der Messpunkte pro Kanton
df_hail_canton = df_hail_canton.merge(points_per_canton[["canton", "lat_lon"]], left_on="canton", right_on="canton")
# Normierung der Summe der Hageltage pro Jahr und Kanton auf die Anzahl der Messpunkte im Gitter pro Kanton
df_hail_canton["haildays_per_point"] = df_hail_canton.haildays_sum / df_hail_canton.lat_lon
df_hail_canton.head()

In [ ]:
# join Hageldaten pro Kanton und Jahr mit Kanton Shapefile
canton_data_haildays = canton_map.merge(df_hail_canton, left_on="NAME", right_on="canton")

#### Imagegrid zur Anzahl der Hageltage pro Jahr und Kanton von 2002 - 2022

In [ ]:
# ImageGrid 7x3 erstellen für die Jahre 2002-2022
fig = plt.figure(figsize=(18, 18))
grid = ImageGrid(fig, 111, nrows_ncols=(7, 3), axes_pad=0.1)

# Colormap Skala Definitionen (Maximum und Minimum definiert über alle Jahre)
min_haildays = canton_data_haildays["haildays_per_point"].min()
max_haildays = canton_data_haildays["haildays_per_point"].max()

# ImageGrid füllen (Schleife über alle Jahre)
for i, year in enumerate(years):
    year_data = canton_data_haildays[canton_data_haildays["year"] == year]
    ax = grid[i]
    ax.axis("off")
    ax.set_title(year)
    year_data.plot(
        column="haildays_per_point",
        cmap=color_map_haildays,
        linewidth=0.1,
        ax=ax, 
        legend=False,
        vmin=min_haildays,
        vmax=max_haildays,
    )

# Hinzufügen einer Colorbar
ax = grid[0]
sm = plt.cm.ScalarMappable(
    cmap=color_map_haildays,
    norm=plt.Normalize(vmin=min_haildays, vmax=max_haildays),
)
cbar = plt.colorbar(sm, ax=ax, pad=2, aspect=40)
cbar.ax.tick_params(labelsize=12)
cbar.ax.set_ylabel(r"$\Sigma$ Hageltage / Anzahl Messpunkte Kanton", rotation=270, labelpad=20, size=14)

# Titel hinzufügen
fig.suptitle("Summe Hageltage normiert auf Anzahl der Messpunkte pro Kanton", fontsize=18, y=0.92, x=0.55)

plt.show()

#### Interkativer Plot zur Anzahl der Hageltage pro Jahr und Kanton von 2002 - 2022

In [ ]:
plt.rcParams["figure.figsize"] = (12, 8)

# Colormap Skala Definitionen (Maximum und Minimum definiert über alle Jahre)
min_haildays = canton_data_haildays["haildays_per_point"].min()
max_haildays = canton_data_haildays["haildays_per_point"].max()

# Defintion der Plot Update Funktion für die Jahresauswahl
def update_plot(year: int) -> None:
    # Filterung der Daten nach Jahresauswahl
    year_data = canton_data_haildays[canton_data_haildays["year"] == year]
    ax = year_data.plot(
        column="haildays_per_point",
        cmap=color_map_haildays,
        linewidth=0.1,
        legend=False,
        vmin=min_haildays,
        vmax=max_haildays,
    )
    ax.axis("off")
    ax.set_title(f"Summe Hageltage normiert auf Anzahl der Messpunkte pro Kanton {year}")
    # Axes Objekt von GeoAxesSubplot Objekt extrahieren
    ax = ax.axes
    # erzeugen und hinzufügen einer Colorbar
    sm = plt.cm.ScalarMappable(
        cmap=color_map_haildays,
        norm=plt.Normalize(vmin=min_haildays, vmax=max_haildays),
    )
    cbar = plt.colorbar(sm, ax=ax, fraction=0.05, pad=0.03)
    cbar.ax.set_ylabel(r"$\Sigma$ Hageltage / Anzahl Messpunkte Kanton", rotation=270, labelpad=20)
    plt.show()


# interaktiver Slider
interactive_plot_haildays = interactive(update_plot, year=years, continuous_update=False)

# interaktiven Plot darstellen
display(interactive_plot_haildays)

#### Verlauf der Summe der Hageltage je Kanton und Jahr normiert auf die Anzahl der Messpunkte je Kanton

In [ ]:
fig = px.line(df_hail_canton, x="year", y="haildays_per_point", color="canton")

fig.update_layout(
    title="Verlauf der Summe der Hageltage je Kanton und Jahr normiert auf die Anzahl der Messpunkte je Kanton",
    xaxis_title="Jahr",
    yaxis_title="Summe Hageltage pro Messpunkt",
    legend_title="Kanton",
)
fig.show(config= dict(displayModeBar = False))

In [ ]:
# Erzeugen eines Dataframes für die gesamte Schweiz basierend auf obigen Dataframe für Summe der Hageltage pro Jahr und Kanton
df_haildays_total = df_hail_canton.groupby("year").agg({"haildays_sum": sum, "lat_lon": sum}).reset_index()
df_haildays_total["haildays_per_point"] = df_haildays_total["haildays_sum"] / df_haildays_total["lat_lon"]
df_haildays_total["canton"] = "All"
df_haildays_total.head()

#### Verlauf der Summe der Hageltage pro Jahr normiert auf die Anzahl der Messpunkte in der Schweiz

In [ ]:
fig = px.line(df_haildays_total, x="year", y="haildays_per_point")
fig.update_layout(
    title="Verlauf der Summe der Hageltage pro Jahr normiert auf die Anzahl der Messpunkte in der Schweiz",
    xaxis_title="Jahr",
    yaxis_title="Hageltage pro Messpunkt",
    legend_title="Kanton",
)

fig.show(config= dict(displayModeBar = False))

### 2.3.2 Mittlere maximal zu erwartende Hagelkorngrösse pro Jahr und Kanton

In [ ]:
# Gruppieren der Hageldaten nach Jahr und Kanton und Aufsummierung der Hagelkorngrösse
df_hailsize_canton = df_hailsize.groupby(["year", "canton"]).agg(
    hailsize_sum = ("hailsize", "sum"),
).reset_index()

# Zusammenführen der Hageldaten nach Jahr und Kanton mit Anzahl der Messpunkte pro Kanton
df_hailsize_canton = df_hailsize_canton.merge(points_per_canton[["canton", "lat_lon"]], left_on="canton", right_on="canton")
# Normierung der Summe der Hageltage pro Jahr und Kanton auf die Anzahl der Messpunkte im Gitter pro Kanton
df_hailsize_canton["hailsize_per_point"] = df_hailsize_canton.hailsize_sum / df_hailsize_canton.lat_lon
df_hailsize_canton.head()

In [ ]:
# join Hageldaten pro Kanton und Jahr mit Kanton Shapefile
canton_data_hailsize = canton_map.merge(df_hailsize_canton, left_on="NAME", right_on="canton")

#### Imagegrid zur maximal zu erwartende Hagelkorngrösse pro Jahr und Kanton von 2002 - 2022

In [ ]:
# ImageGrid 7x3 erstellen für die Jahre 2002-2022
fig = plt.figure(figsize=(18, 18))
grid = ImageGrid(fig, 111, nrows_ncols=(7, 3), axes_pad=0.1)

# Colormap Skala Definitionen (Maximum und Minimum definiert über alle Jahre)
min_hailsize = canton_data_hailsize["hailsize_per_point"].min()
max_hailsize = canton_data_hailsize["hailsize_per_point"].max()

# ImageGrid füllen
for i, year in enumerate(years):
    year_data = canton_data_hailsize[canton_data_hailsize["year"] == year]
    ax = grid[i]
    ax.axis("off")
    ax.set_title(year)
    year_data.plot(
        column="hailsize_per_point",
        cmap=color_map_hailsize,
        linewidth=0.1,
        ax=ax, 
        legend=False,
        vmin=min_hailsize,
        vmax=max_hailsize,
    )

# Hinzufügen einer Colorbar
ax = grid[0]
sm = plt.cm.ScalarMappable(
    cmap=color_map_hailsize,
    norm=plt.Normalize(vmin=min_hailsize, vmax=max_hailsize),
)
cbar = plt.colorbar(sm, ax=ax, pad=2, aspect=40)
cbar.ax.tick_params(labelsize=12)
cbar.ax.set_ylabel(r"$\Sigma$ maximale Hagelkorngrösse [cm] / Anzahl Messpunkte Kanton", rotation=270, labelpad=20, size=14)

# Titel hinzufügen
fig.suptitle("Summe maximale Hagelkorngrösse in cm normiert auf Anzahl der Messpunkte pro Kanton", fontsize=18, y=0.92, x=0.55)

plt.show()

#### Interkativer Plot zur maximal zu erwartende Hagelkorngrösse pro Jahr und Kanton von 2002 - 2022

In [ ]:
plt.rcParams["figure.figsize"] = (12, 8)

# Colormap Skala Definitionen (Maximum und Minimum definiert über alle Jahre)
min_hailsize = canton_data_hailsize["hailsize_per_point"].min()
max_hailsize = canton_data_hailsize["hailsize_per_point"].max()

# Defintion der Plot Update Funktion für die Jahresauswahl
def update_plot(year):
    # Filterung der Daten nach Jahresauswahl
    year_data = canton_data_hailsize[canton_data_hailsize["year"] == year]
    ax = year_data.plot(
        column="hailsize_per_point",
        cmap=color_map_hailsize,
        linewidth=0.1,
        legend=False,
        vmin=min_hailsize,
        vmax=max_hailsize,
    )
    ax.axis("off")
    ax.set_title("Summe maximale Hagelkorngrösse in cm normiert auf Messpunkte je Kanton" + f" {year}")
    # Axes Objekt von GeoAxesSubplot Objekt extrahieren
    ax = ax.axes
    # erzeugen und hinzufügen einer Colorbar
    sm = plt.cm.ScalarMappable(
        cmap=color_map_hailsize,
        norm=plt.Normalize(vmin=min_hailsize, vmax=max_hailsize),
    )
    cbar = plt.colorbar(sm, ax=ax, fraction=0.05, pad=0.03)
    cbar.ax.set_ylabel(r"$\Sigma$ maximale Hagelkorngrösse [cm] / Anzahl Messpunkte", rotation=270, labelpad=20)
    plt.show()

    
# interaktiver Slider
interactive_plot = interactive(update_plot, year=years, continuous_update=False)

# interaktiven Plot darstellen
display(interactive_plot)

#### Verlauf der Summe der maximal zu erwartenden Hagelkorngrösse in cm je Kanton und Jahr normiert auf die Anzahl der Messpunkte je Kanton

In [ ]:
fig = px.line(df_hailsize_canton, x="year", y="hailsize_per_point", color="canton")

fig.update_layout(
    title="Verlauf der Summe der maximal zu erwartenden Hagelkorngrösse in cm je Kanton und Jahr normiert auf die Anzahl der Messpunkte je Kanton",
    xaxis_title="Jahr",
    yaxis_title="Summe Hagelkorngrösse in cm pro Messpunkt",
    legend_title="Kanton",
)
fig.show(config= dict(displayModeBar = False))

In [ ]:
# Erzeugen eines Dataframes für die gesamte Schweiz basierend auf obigen Dataframe für Summe der Hagelkorngrösse pro Jahr und Kanton
df_hailsize_total = df_hailsize_canton.groupby("year").agg({"hailsize_sum": sum, "lat_lon": sum}).reset_index()
df_hailsize_total["hailsize_per_point"] = df_hailsize_total["hailsize_sum"] / df_hailsize_total["lat_lon"]
df_hailsize_total["canton"] = "All"
df_hailsize_total.head()

#### Verlauf der Summe der maximal zu erwartenden Hagelkorngrösse in cm pro Jahr normiert auf die Anzahl der Messpunkte in der Schweiz

In [ ]:
fig = px.line(df_hailsize_total, x="year", y="hailsize_per_point")
fig.update_layout(
    title="Verlauf der Summe der maximal zu erwartenden Hagelkorngrösse in cm pro Jahr normiert auf die Anzahl der Messpunkte in der Schweiz",
    xaxis_title="Jahr",
    yaxis_title="Summe Hagelkorngrösse in cm pro Messpunkt",
)

fig.show(config= dict(displayModeBar = False))

<a id="hail_canton_month"></a>
## 2.4 Hagelrisiko nach Monat und Kantone

Im folgenden Abschnitt wird das Hagelrisiko nach Monat und Kanton analysiert und visualisiert. Dabei werden die Daten zu den Hageltage und die maximal zu erwartenden Hagelkorngrösse von 2002 bis 2022 verwendet.

In [ ]:
df_hail.head(n=2)

### 2.4.1 Summe der Hageltage pro Monat und Kanton

In [ ]:
# Gruppieren der Hageldaten nach Monat und Jahr und Aufsummierung der Hageltage
df_hail_month_year = df_hail.groupby(["month", "year"]).agg(
    haildays_sum = ("haildays", "sum"),
).reset_index()
# Umbenennung der 'year' und 'month' Spalte in deutsche Namen für sprachlich konsistente Plots
df_hail_month_year = df_hail_month_year.rename(
    columns={
        "year": "Jahr",
        "month": "Monat",
    }
)

# Normierung auf die Anzahl der Messpunkte in der Schweiz
df_hail_month_year["haildays_sum"] /= points_switzerland

df_hail_month_year.head()

#### Summe der Hageltage pro Monat von 2002-2022

In [ ]:
fig = px.bar(df_hail_month_year, x="Monat", y="haildays_sum", color="Jahr")

fig.update_layout(
    title="Summe der Hageltage pro Monat von 2002-2022",
    xaxis_title="Monat",
    xaxis=dict(
        tickmode="array",
        tickvals=[4, 5, 6, 7, 8, 9],
        ticktext=["April", "Mai", "Juni", "Juli", "August", "September"],
    ),
    yaxis_title="Summe Hageltage / Anzahl Messpunkte",
)

fig.show(config= dict(displayModeBar = False))

In [ ]:
# Gruppieren der Hageldaten nach Kanton und Aufsummierung der Hageltage
df_hail_canton_total = df_hail.groupby("canton").agg(
    haildays_sum = ("haildays", "sum"),
).reset_index()

# Zusammenführen der Hageltage nach Kanton mit Messpunkten im Gitter je Kanton
df_hail_canton_total = df_hail_canton_total.merge(
    points_per_canton[["canton", "lat_lon"]],
    left_on="canton",
    right_on="canton",
)

# Normierung der Summe der Hageltage je Kanton auf Messpunkte pro Kanton
df_hail_canton_total["haildays_per_point"] = (
    df_hail_canton_total["haildays_sum"]
    / df_hail_canton_total["lat_lon"]
    / len(years)
)
df_hail_canton_total.head()

#### Anteil an Summe der Hageltage 2002-2022 pro Kanton normiert auf die Anzahl der Messpunkte je Kanton

In [ ]:
fig = px.pie(
    df_hail_canton_total,
    values="haildays_per_point",
    names="canton",
    hole=.3,
    color_discrete_sequence=px.colors.sequential.YlGnBu
)

fig.update_layout(
    title={
        "text": "Anteil an Summe der Hageltage 2002-2022 pro Kanton normiert auf die Anzahl der Messpunkte je Kanton",
        "y": 0.1,
        "x": 0.4,
        "xanchor": "center",
        "yanchor": "top",
    },
    legend_title="Kanton",
)

fig.show(config= dict(displayModeBar = False))

In [ ]:
# Gruppieren der Hageldaten nach Messzeitpunkt und Kanton und Aufsummierung der Hageltage
df_hail_date = df_hail.groupby(["time", "canton"]).agg(
    haildays_sum = ("haildays", "sum"),
).reset_index()

# Zusammenführen des obigen Dataframes mit Messpunkten im Gitter je Kanton
df_hail_date = df_hail_date.merge(
    points_per_canton[["canton", "lat_lon"]],
    left_on="canton",
    right_on="canton",
)

# Normierung der Summe der Hageltage je Zeitpunkt und Kanton auf Messpunkte pro Kanton
df_hail_date["haildays_per_point"] = df_hail_date.haildays_sum / df_hail_date.lat_lon

df_hail_date.head()

#### Summe der Hageltage pro Monat und Jahr je Kanton

In [ ]:
fig = px.line(df_hail_date, x="time", y="haildays_sum", color="canton")

fig.update_layout(
    title="Summe der Hageltage pro Monat und Jahr je Kanton",
    xaxis_title="Monat",
    yaxis_title="Summe Hageltage",
    legend_title="Kanton",
)
fig.show(config= dict(displayModeBar = False))

In [ ]:
# Gruppieren der Hageldaten nach Messzeitpunkt und Aufsummierung der Hageltage
df_hail_time_total = df_hail.groupby(["time"]).agg(
    haildays_sum = ("haildays", "sum"),
).reset_index()

# Normierung der Summe der Hageltage je Zeitpunkt auf Anzahl der Messpunkte in der Schweiz
df_hail_time_total["haildays_per_point"] = (
    df_hail_time_total.haildays_sum
    / points_switzerland
)
df_hail_time_total.head()

#### Summe der Hageltage pro Monat und Jahr normiert auf Anzahl der Messpunkte

In [ ]:
fig = px.line(df_hail_time_total, x="time", y="haildays_per_point")
fig.update_layout(
    title="Summe der Hageltage pro Monat und Jahr normiert auf Anzahl der Messpunkte",
    xaxis_title="Monat",
    yaxis_title="Summe Hageltage pro Messpunkt",
)
fig.show(config= dict(displayModeBar = False))

### 2.4.2 Hagelkorngrösse pro Monat je Kanton

In [ ]:
# Gruppieren der Hageldaten nach Monat und Jahr und Aufsummierung der Hageltage und Produkt der Hageltage u. Hagelkorngrösse
df_hailsize_month_year = df_hailsize.groupby(["month", "year"]).agg(
    hailsize_sum = ("haildays_times_hailsize", "sum"),
    haildays_sum = ("haildays", "sum"),
).reset_index()

# Umbenennung der 'year' und 'month' Spalte in deutsche Namen für sprachlich konsistente Plots
df_hailsize_month_year = df_hailsize_month_year.rename(
    columns={
        "year": "Jahr",
        "month": "Monat",
    }
)

# Bestimmung der mittleren maximal zu erwartenden Hagelkorngrösse je Monat und Jahr
df_hailsize_month_year["hailsize_mean"] = (
    df_hailsize_month_year["hailsize_sum"]
    / df_hailsize_month_year["haildays_sum"]
)

df_hailsize_month_year.head()

#### Mittlere maximale Hagelkorngrösse in cm pro Monat von 2002-2022

In [ ]:
fig = px.bar(df_hailsize_month_year, x="Monat", y="hailsize_mean", color="Jahr")

fig.update_layout(
    title="Mittlere maximale Hagelkorngrösse in cm pro Monat von 2002-2022",
    xaxis_title="Monat",
    xaxis=dict(
        tickmode="array",
        tickvals=[4, 5, 6, 7, 8, 9],
        ticktext=["April", "Mai", "Juni", "Juli", "August", "September"],
    ),
    yaxis_title="Mittlere maximale Hagelkorngrösse [cm]",
)

fig.show(config= dict(displayModeBar = False))

In [ ]:
# Gruppieren der Hageldaten nach Kanton und Aufsummierung des Produktes aus Hageltage und Hagelkorngrösse
df_hailsize_canton_total = df_hailsize.groupby("canton").agg(
    hailsize_sum = ("haildays_times_hailsize", "sum"),
).reset_index()

# Zusammenführn des obigen Dataframes mit Anzahl der Messpuntke pro Kanton
df_hailsize_canton_total = df_hailsize_canton_total.merge(
    points_per_canton[["canton", "lat_lon"]],
    left_on="canton",
    right_on="canton",
)

# Normierung der maximal zu erwartenden Hagelkorngrösse je Kanton auf Anzahl der Messpunkte pro Kanton
# und Anzhal der Messjahre (2002-2022 -> 21 Jahre)
df_hailsize_canton_total["hailsize_per_point"] = (
    df_hailsize_canton_total["hailsize_sum"]
    / df_hailsize_canton_total["lat_lon"]
    / len(years)
)
df_hailsize_canton_total.head()

#### Mittlere maximale Hagelkorngrösse von 2002-2022 pro Kanton normiert auf die Anzahl der Messpunkte je Kanton

In [ ]:
fig = px.pie(
    df_hailsize_canton_total,
    values="hailsize_per_point",
    names="canton",
    hole=.3,
    color_discrete_sequence=px.colors.sequential.YlOrRd
)

fig.update_layout(
    title={
        "text": "Mittlere maximale Hagelkorngrösse von 2002-2022 pro Kanton normiert auf die Anzahl der Messpunkte je Kanton",
        "y": 0.1,
        "x": 0.4,
        "xanchor": "center",
        "yanchor": "top",
    },
    legend_title="Kanton",
)

fig.show(config= dict(displayModeBar = False))

In [ ]:
# Gruppieren der Hageldaten nach Messzeitpunkt und Kanton und Aufsummierung des Produktes aus Hageltage und Hagelkorngrösse
df_hailsize_date = df_hailsize.groupby(["time", "canton"]).agg(
    hailsize_sum = ("haildays_times_hailsize", "sum"),
).reset_index()

# Zusammenführen des obigen Dataframes mit Anzahl der Messpunkte je Kanton
df_hailsize_date = df_hailsize_date.merge(
    points_per_canton[["canton", "lat_lon"]],
    left_on="canton",
    right_on="canton",
)

# Normierung der Summe der maximal zu erwartenden Hagelkorngrösse je Kanton und Messzeitpunkt
# auf Anzahl der Messpunkte pro Kanton
df_hailsize_date["hailsize_per_point"] = (
    df_hailsize_date.hailsize_sum
    / df_hailsize_date.lat_lon
)

df_hailsize_date.head()

#### Mittlere maximale Hagelkorngrösse in cm pro Monat und Jahr je Kanton

In [ ]:
fig = px.line(df_hailsize_date, x="time", y="hailsize_per_point", color="canton")

fig.update_layout(
    title="Mittlere maximale Hagelkorngrösse in cm pro Monat und Jahr je Kanton",
    xaxis_title="Monat",
    yaxis_title="Mittlere maximale Hagelkorngrösse [cm]",
    legend_title="Kanton",
)
fig.show(config= dict(displayModeBar = False))

In [ ]:
# Gruppieren der Hageldaten nach Messzeitpunkt und Aufsummierung des Produktes aus Hageltage und Hagelkorngrösse
df_hailsize_time_total = df_hailsize.groupby(["time"]).agg(
    hailsize_sum = ("haildays_times_hailsize", "sum"),
).reset_index()

# Normierung des Summe der Hagelkrongrösse auf Anzahl der Messpunkte in der Schweiz
df_hailsize_time_total["hailsize_per_point"] = (
    df_hailsize_time_total.hailsize_sum
    / points_switzerland
)
df_hailsize_time_total.head()

#### Mittlere maximale Hagelkorngrösse in cm pro Monat und Jahr normiert auf Anzahl der Messpunkte in der Schweiz

In [ ]:
fig = px.line(df_hail_time_total, x="time", y="haildays_per_point")
fig.update_layout(
    title="Mittlere maximale Hagelkorngrösse in cm pro Monat und Jahr normiert auf Anzahl der Messpunkte in der Schweiz",
    xaxis_title="Monat",
    yaxis_title="Mittlere maximale Hagelkorngrösse [cm]",
)
fig.show(config= dict(displayModeBar = False))